In [1]:
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import random
import time
import pandas as pd
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import hamming
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
import pandas as pd
df = pd.read_excel('bert_training_data.xlsx')
df['user_defined_input'] = df['user_defined_input'].str.split('\n')

# Creating a new row for each description
df_exploded = df.explode('user_defined_input')
df = df_exploded
df['symptom'] = df['symptom'].str.strip()


In [4]:
df

,symptom,user_defined_input
0,ulcers_on_tongue,I woke up this morning with a painful sore on ...
0,ulcers_on_tongue,My tongue feels irritated and there's a small ...
0,ulcers_on_tongue,It hurts to stick my tongue out - there's a re...
0,ulcers_on_tongue,"Lately, I've had these recurring mouth ulcers ..."
0,ulcers_on_tongue,Something feels weird on my tongue - it's like...
...,...,...
130,weakness_in_limbs,"Doctor, I've been experiencing weakness and fa..."
130,weakness_in_limbs,There's this heaviness and lack of strength in...
130,weakness_in_limbs,My weakness in limbs seems to worsen with phys...
130,weakness_in_limbs,I woke up this morning feeling weak and unstea...


In [ ]:
class_counts = df['symptom'].value_counts()

# Define the minimum number of instances required per class
min_instances = 2

# Filter out classes with fewer than the minimum required number of instances
filtered_classes = class_counts[class_counts >= min_instances].index.tolist()

# Keep only the data with classes that meet the minimum instance criterion
df_filtered = df[df['symptom'].isin(filtered_classes)]


## Finetuning and saving bert model

In [ ]:


# Perform the train-test split to create training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    df_filtered['user_defined_input'],
    df_filtered['symptom'],
    test_size=0.10,  # Represents 10% of the dataset for validation
    random_state=42,
    stratify=df_filtered['symptom']  # Stratify by the symptom labels to maintain distribution
)



In [ ]:
train_data

,description,symptom
16,"Doctor, could my muscle weakness be a sign of ...",muscle_weakness
16,"It's like there's no power left in my muscles,...",muscle_weakness
12,I woke up this morning with this sharp pain in...,bladder_discomfort
32,Stress eating is a bad habit! I've gained a fe...,weight_gain
40,"Doctor, could my painful walking be a sign of ...",painful_walking
...,...,...
7,I keep clearing my throat because it feels so ...,throat_irritation
129,"Doctor, I've been experiencing hip joint pain ...",hip_joint_pain
25,I've been avoiding social situations because o...,pus_filled_pimples
114,"Doctor, I've noticed some small dents or pits ...",small_dents_in_nails


In [ ]:

# tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased', do_lower_case=True)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)


In [ ]:
label_dict = {label: idx for idx, label in enumerate(df_filtered['symptom'].unique())}
label_dict

{'ulcers_on_tongue': 0,
 'loss_of_smell': 1,
 'abdominal_pain': 2,
 'sunken_eyes': 3,
 'cough': 4,
 'internal_itching': 5,
 'dehydration': 6,
 'throat_irritation': 7,
 'mucoid_sputum': 8,
 'itching': 9,
 'depression': 10,
 'coma': 11,
 'bladder_discomfort': 12,
 'silver_like_dusting': 13,
 'scurring': 14,
 'lethargy': 15,
 'muscle_weakness': 16,
 'stomach_pain': 17,
 'altered_sensorium': 18,
 'diarrhoea': 19,
 'brittle_nails': 20,
 'puffy_face_and_eyes': 21,
 'runny_nose': 22,
 'slurred_speech': 23,
 'red_sore_around_nose': 24,
 'pus_filled_pimples': 25,
 'watering_from_eyes': 26,
 'belly_pain': 27,
 'nodal_skin_eruptions': 28,
 'congestion': 29,
 'weakness_of_one_body_side': 30,
 'fast_heart_rate': 31,
 'weight_gain': 32,
 'vomiting': 33,
 'irritability': 34,
 'rusty_sputum': 35,
 'distention_of_abdomen': 36,
 'increased_appetite': 37,
 'palpitations': 38,
 'bloody_stool': 39,
 'painful_walking': 40,
 'fatigue': 41,
 'loss_of_appetite': 42,
 'toxic_look_(typhos)': 43,
 'patches_in_thr

In [ ]:
from sklearn.metrics import accuracy_score


# Function to set seed for reproducibility
def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

set_seed(42) # Set your seed value for reproducibility



encoded_data_train = tokenizer.batch_encode_plus(
    X_train.tolist(),
    add_special_tokens=True,
    return_attention_mask=True,
    padding='longest',
    truncation=True,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    X_val.tolist(),
    add_special_tokens=True,
    return_attention_mask=True,
    padding='longest',
    truncation=True,
    return_tensors='pt'
)

# Convert labels to integers
labels_train = torch.tensor([label_dict[label] for label in y_train.tolist()])
labels_val = torch.tensor([label_dict[label] for label in y_val.tolist()])

# Create TensorDatasets
dataset_train = TensorDataset(encoded_data_train['input_ids'], encoded_data_train['attention_mask'], labels_train)
dataset_val = TensorDataset(encoded_data_val['input_ids'], encoded_data_val['attention_mask'], labels_val)

# Data Loaders
batch_size = 32
dataloader_train = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batch_size)
dataloader_validation = DataLoader(dataset_val, sampler=SequentialSampler(dataset_val), batch_size=batch_size)



model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False
)

# Set up optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)
epochs = 50
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader_train) * epochs)

# Define the training loop
def train(dataloader_train):
    model.train()
    total_loss, total_accuracy = 0, 0

    for batch in tqdm(dataloader_train, total=len(dataloader_train)):
        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}

        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataloader_train)
    return average_loss

# Define the evaluation loop
def evaluate(dataloader_val):
    model.eval()
    val_loss, val_accuracy = 0, 0
    predictions, true_vals = [], []

    for batch in tqdm(dataloader_val, total=len(dataloader_val)):
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs.loss
        logits = outputs.logits
        val_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    val_loss = val_loss / len(dataloader_val)
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return val_loss, predictions, true_vals

# Check if GPU is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Run training and validation
for epoch in range(1, epochs + 1):
    train_loss = train(dataloader_train)
    val_loss, predictions, true_vals = evaluate(dataloader_validation)


    print(f'Epoch {epoch} | Train Loss: {train_loss} | Val Loss: {val_loss}')

torch.save(model.state_dict(), 'bert_finetuned.pth')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/111 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 1 | Train Loss: 4.874503294626872 | Val Loss: 4.848720346178327


  0%|          | 0/111 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 49 | Train Loss: 0.13879131401578584 | Val Loss: 1.0645850385938371


  0%|          | 0/111 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 50 | Train Loss: 0.1362077934784932 | Val Loss: 1.0647649424416679


## Running and testing saved model

In [ ]:
label_dict = {label: idx for idx, label in enumerate(df_filtered['symptom'].unique())}

In [ ]:
import spacy

# Load English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")


def process_text(text):
  doc = nlp(text)

  processed_text = " ".join([token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct])
  return processed_text


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False
)

# Load the model's state_dict
model.load_state_dict(torch.load('bert_finetuned.pth', map_location=device))
model.to(device)

def predict_top_symptoms(user_input, top_k=3):
    model.eval()
    user_input = process_text(user_input)
    encoded_input = tokenizer.encode_plus(
        user_input,
        add_special_tokens=True,
        return_attention_mask=True,
        padding='max_length',
        max_length=25,
        truncation=True,
        return_tensors='pt'
    ).to(device)


    # Perform the prediction
    with torch.no_grad():
        output = model(**{
            'input_ids': encoded_input['input_ids'],
            'attention_mask': encoded_input['attention_mask']
        })

    probs = torch.nn.functional.softmax(output.logits, dim=1)

    # Get the top k predicted classes and their probabilities
    top_probs, top_preds = torch.topk(probs, top_k)
    top_probs = top_probs.cpu().numpy()[0]
    top_preds = top_preds.cpu().numpy()[0]

    # Map the top predicted indices back to the symptom names
    top_symptoms = [list(label_dict.keys())[list(label_dict.values()).index(pred)] for pred in top_preds]

    # Combine the symptoms and their probabilities into a list of tuples
    top_symptoms_with_probs = list(zip(top_symptoms, top_probs))

    return top_symptoms_with_probs



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
preds = []
for i in test_data['predicted_symptom_bert']:
  list_preds = []
  for j in predict_top_symptoms(i):
    if j[1]>= 0.05:
      list_preds.append(j[0])

  preds.append(list_preds)
  # print(list_preds)

In [ ]:
preds[:10]

[['breathlessness'],
 ['visual_disturbances', 'spinning_movements'],
 ['knee_pain'],
 ['stomach_bleeding', 'continuous_sneezing', 'blood_in_sputum'],
 ['joint_pain', 'cramps'],
 ['breathlessness', 'painful_walking', 'unsteadiness'],
 ['restlessness'],
 ['dehydration'],
 ['scurring'],
 ['yellowish_skin']]

In [ ]:
len(preds)

196

In [ ]:
# test_data['description'] = [0][0]

test_data['predicted_symptom_bert'] = test_data['description'].apply(predict_top_symptoms)

In [ ]:
accuracy = accuracy_score(test_data['symptom'], test_data['predicted_symptom_bert'])
accuracy

0.6173469387755102

In [ ]:
test_data.to_csv('/content/drive/MyDrive/Spring 2024/CSCE 670 - Info Retrieval/PROJECT/test_data_predictions.csv',index=False)

### Experimetng with predict_top_symptoms function

In [ ]:

user_input = "I have a rash."
top_symptoms_with_probs = predict_top_symptoms(user_input, top_k=5)[0][0]

print(f"Top Predicted Symptoms and Probabilities: {top_symptoms_with_probs}")


Top Predicted Symptoms and Probabilities: mood_swings


In [ ]:
user_input = "I'm not able to stop sneezing and I'm getting cold and shivering. Also, I'm getting water from my eyes"
top_symptoms_with_probs = predict_top_symptoms(user_input, top_k=8)
print(f"Top Predicted Symptoms and Probabilities: {top_symptoms_with_probs}")

Top Predicted Symptoms and Probabilities: [('continuous_sneezing', 0.96464634), ('runny_nose', 0.0018105542), ('cough', 0.0014213452), ('vomiting', 0.00107308), ('throat_irritation', 0.0008678902), ('stomach_bleeding', 0.00078919734), ('scurring', 0.00077170046), ('dischromic _patches', 0.00067294115)]


In [ ]:

user_input = "I'm not able to stop sneezing and I'm getting cold and shivering. Also, I'm getting water from my eyes"
top_symptoms_with_probs = predict_top_symptoms(user_input, top_k=8)
print(f"Top Predicted Symptoms and Probabilities: {top_symptoms_with_probs}")


able stop sneeze get cold shivering get water eye
Top Predicted Symptoms and Probabilities: [('continuous_sneezing', 0.56420374), ('runny_nose', 0.027326934), ('vomiting', 0.020540215), ('cough', 0.017793637), ('throat_irritation', 0.011908777), ('scurring', 0.011072687), ('rusty_sputum', 0.010546043), ('mucoid_sputum', 0.008553919)]


In [ ]:
user_input = "heart is beating really fast"
top_symptoms_with_probs = predict_top_symptoms(user_input, top_k=5)

print(f"Top Predicted Symptoms and Probabilities: {top_symptoms_with_probs}")

Top Predicted Symptoms and Probabilities: [('fast_heart_rate', 0.8262287), ('palpitations', 0.06256688), ('stomach_bleeding', 0.005310559), ('chest_pain', 0.0043458277), ('mood_swings', 0.003927628)]


In [ ]:
user_input = "heart is beating really fast"
top_symptoms_with_probs = predict_top_symptoms(user_input, top_k=5)

print(f"Top Predicted Symptoms and Probabilities: {top_symptoms_with_probs}")

heart beat fast
Top Predicted Symptoms and Probabilities: [('fast_heart_rate', 0.4903701), ('palpitations', 0.22401237), ('extra_marital_contacts', 0.006283723), ('acute_liver_failure', 0.0056153154), ('chest_pain', 0.005046033)]


In [ ]:
user_input = "i cant breathe"
top_symptoms_with_probs = predict_top_symptoms(user_input, top_k=5)

print(f"Top Predicted Symptoms and Probabilities: {top_symptoms_with_probs}")

not breathe
Top Predicted Symptoms and Probabilities: [('restlessness', 0.12582229), ('breathlessness', 0.09983016), ('anxiety', 0.037642088), ('lack_of_concentration', 0.035046212), ('loss_of_balance', 0.028838148)]


In [ ]:
user_input = "I'm having fever for last 2 days and I have a sore throat"
top_symptoms_with_probs = predict_top_symptoms(user_input, top_k=5)
print(f"Top Predicted Symptoms and Probabilities: {top_symptoms_with_probs}")

have fever 2 day sore throat
Top Predicted Symptoms and Probabilities: [('throat_irritation', 0.2280863), ('cough', 0.09652748), ('patches_in_throat', 0.07680216), ('mild_fever', 0.056921374), ('vomiting', 0.043752167)]


In [ ]:
user_input = "my body is itchy and i have a sore throat"
top_symptoms_with_probs = predict_top_symptoms(user_input, top_k=5)

print(f"Top Predicted Symptoms and Probabilities: {top_symptoms_with_probs}")

body itchy sore throat
Top Predicted Symptoms and Probabilities: [('throat_irritation', 0.3629192), ('patches_in_throat', 0.22240986), ('cough', 0.033046775), ('rusty_sputum', 0.014615109), ('swelled_lymph_nodes', 0.009515931)]


In [ ]:
user_input = "im feeling something weird in my chest"
top_symptoms_with_probs = predict_top_symptoms(user_input, top_k=5)

print(f"Top Predicted Symptoms and Probabilities: {top_symptoms_with_probs}")

m feel weird chest
Top Predicted Symptoms and Probabilities: [('chest_pain', 0.11941125), ('anxiety', 0.07590945), ('altered_sensorium', 0.039685857), ('mood_swings', 0.03811604), ('belly_pain', 0.030911729)]


### Integrating with user-user CF

In [ ]:
data_w_disease = pd.read_csv('processed_data.csv')

data = data_w_disease.drop('Disease', axis=1)
data_rec = data.drop(columns=['user_id'])

col_in_data = data_rec.columns

def get_symptoms(top_symptoms_with_probs):
    temp_col = []
    max_value = 0
    max_symp = ''

    # Iterate over symptoms to find column names
    for symptom, value in top_symptoms_with_probs:
        if value >= 0.09:
            temp_col.append(symptom)
        elif value >= max_value:
            max_value = value
            max_symp = symptom

    if len(temp_col) == 0:
        temp_col.append(max_symp)

    return temp_col

In [ ]:
def predict_symptoms(user_input):
    top_symptoms_with_probs = predict_top_symptoms(user_input, top_k=5)

    return get_symptoms(top_symptoms_with_probs)



In [ ]:
df = pd.read_csv('DiseasePrecaution.csv')
df.replace("NaN", pd.NA, inplace=True)


precautions_dict = {row['Disease']: [value for value in row[1:] if pd.notna(value)] for index, row in df.iterrows()}


In [ ]:
from collections import Counter



# The list user_inputs should have all the text provided by the user

def final_input(user_inputs):
  column_names = []

  for input in user_inputs:
      column_names += predict_symptoms(input)

  data_dict = {}

  for col_name in col_in_data:
    if col_name.strip() in column_names:
      data_dict[col_name] = 1
    else:
      data_dict[col_name] = 0
  user_symp = pd.DataFrame([data_dict])


  similarities = cosine_similarity(user_symp, data_rec)

  top_users_indices = np.argsort(similarities[0])[-10:][::-1]

  sim_user_diseases = [data_w_disease.iloc[d, 1] for d in top_users_indices]

  count = Counter(sim_user_diseases)

  # Find the most frequent disease
  diagnosis = count.most_common(1)[0][0]

  print(f"Final Diagnosis: {diagnosis}. To alleviate your symptoms, we recommend:{precautions_dict[diagnosis]} ")

In [ ]:
user_inputs = ["I've headache and feel irritation", "I've acidity but still feel hungry every time",
               "Got blurred vision and disturbances in vision", "I've pain in neck muscles and I feel depressed everytime"]

final_input(user_inputs)

Final Diagnosis: Migraine. To alleviate your symptoms, we recommend:['meditation', 'reduce stress', 'use poloroid glasses in sun', 'consult doctor'] 


In [ ]:
user_inputs = [
    "I have been feeling very thirsty more often than usual.",
    "I've noticed that I need to urinate more frequently, especially at night.",
    "I'm experiencing unexplained weight loss despite not changing my eating habits.",
    "I feel tired all the time now, even when I haven't done much."
]
final_input(user_inputs)

Final Diagnosis: Diabetes . To alleviate your symptoms, we recommend:['have balanced diet', 'exercise', 'consult doctor', 'follow up'] 


In [ ]:
user_inputs = [
    "I have a burning sensation in my chest, especially after eating.",
    "I sometimes taste stomach acid in the back of my throat, especially at night.",
    "I've started having difficulty swallowing my food.",
    "I frequently get a cough or wheeze, which seems worse when I'm lying down."
]

#
final_input(user_inputs)

Final Diagnosis: GERD. To alleviate your symptoms, we recommend:['avoid fatty spicy food', 'avoid lying down after eating', 'maintain healthy weight', 'exercise'] 


In [ ]:
user_inputs = [
    "I frequently have headaches that don't seem to go away.",
    "I've been experiencing episodes of dizziness and occasional fainting.",
    "I sometimes feel a pounding in my chest, neck, or ears.",
    "I noticed that my vision sometimes becomes blurred for no apparent reason."
]
final_input(user_inputs)

Final Diagnosis: Hypertension . To alleviate your symptoms, we recommend:['meditation', 'salt baths', 'reduce stress', 'get proper sleep'] 
